In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import Normalize

In [ ]:
from ipywidgets import interact

In [ ]:
import rmsd as rmsdlib

In [ ]:
rmsdlib.__version__

# Directories

In [ ]:
resources = Path("../tests/resources/").resolve()

# Plot

In [ ]:
import plot_funcs
import coord_funcs

# Read file

In [ ]:
# https://pubchem.ncbi.nlm.nih.gov/compound/15634633#section=Other-Identifiers
filename_a = resources / "examples" / "NS00006345.xyz"
filename_a = resources / "examples" / "CHEMBL135626.xyz"
filename_a = resources / "examples" / "test.xyz"
filename_a = resources / "issue93/d.xyz" 

In [ ]:
atoms_a, coord_a = rmsdlib.get_coordinates_xyz(filename_a, return_atoms_as_int=True)

# Ignore Hydrogens
view_hydrogens, = np.where(atoms_a != 1)

atoms_a = atoms_a[view_hydrogens]
coord_a = coord_a[view_hydrogens]
coord_a -= rmsdlib.get_cm(atoms_a, coord_a)

# Just change the molecule a little bit. Alchemically change Carbon to Nitrogen
# atoms_a[1] = 7 

In [ ]:
fig_width = np.max([np.max(coord_a), -np.min(coord_a)]) * 1.2
fig_width = 3.0

In [ ]:
fig, ax = plot_funcs.get_plot()

print(fig_width)

plot_funcs.plot_coord(ax, atoms_a, coord_a)

ax.set_xlim(-fig_width, fig_width)
ax.set_ylim(-fig_width, fig_width)
pass

In [ ]:
def plot_eig(ax_, atoms_, coord_, recenter=True):

    if recenter:
        coord_ -= rmsdlib.get_cm(atoms_, coord_)
        
    center = rmsdlib.get_cm(atoms_, coord_) # Should be zero

    inertia_ = rmsdlib.get_inertia_tensor(atoms_, coord_)
    eigval_, eigvec_ = np.linalg.eig(inertia_)
    eigvec_ = eigvec_.T
    eigvec_ = eigvec_[np.argsort(eigval_)]
    
    plot_funcs.plot_coord(ax_, atoms_, coord_, show_hydrogens=False)
    plot_funcs.do_dot(ax_, center[0], center[1], size=4)
    ax_.arrow(center[0], center[1], center[0] + eigvec_[0][0], center[1] + eigvec_[0][1], color="red")
    ax_.arrow(center[0], center[1], center[0] + eigvec_[1][0], center[1] + eigvec_[1][1], color="blue")
    return

In [ ]:
def box68(atoms_a, coord_a):
    
    fig, axs = plot_funcs.get_plot(4, size=4)
    ax1, ax2, ax3, ax4 = axs

    ax1.set_xlim(-fig_width, fig_width)
    ax1.set_ylim(-fig_width, fig_width)
    ax1.grid(color='grey', linestyle='-', linewidth=0.5)
    ax2.grid(color='grey', linestyle='-', linewidth=0.5)
    ax3.grid(color='grey', linestyle='-', linewidth=0.5)
    ax4.grid(color='grey', linestyle='-', linewidth=0.5)

    print("A:")
    coord_a -= rmsdlib.get_cm(atoms_a, coord_a)
    plot_eig(ax1, atoms_a, coord_a)

    print("B:")
    U = coord_funcs.rotation_matrix(5)
    _xy = np.dot(coord_a[:,:2], U)
    coord_b = coord_a.copy() # copy?
    coord_b[:,:2] = _xy
    
    coord_b -= rmsdlib.get_cm(atoms_a, coord_b)
    plot_eig(ax2, atoms_a, coord_b)

    inertia_b = rmsdlib.get_inertia_tensor(atoms_a, coord_b)
    eigval_b, eigvec_b = np.linalg.eig(inertia_b)
    
    #eigvec_b = eigvec_b.T
    #eigvec_b = eigvec_b[np.argsort(eigval_b)]

    # TODO Mirror molecule?

    print("C:")
    print("det:", np.linalg.det(eigvec_b))
    
    #if np.linalg.det(eigvec_b) < 0:
    #    eigvec_b[1,:] = -eigvec_b[1,:]
    
    coord_c = np.dot(coord_b, eigvec_b)

    plot_eig(ax3, atoms_a, coord_c)

    return

    

    print("D:")


box68(atoms_a, coord_a)

In [ ]:

def _box15():
    x = np.linspace(-10, 10,100)
    
    def f(x, A, B, C):
        return A*x**2 + B*x + C
    
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    line, = ax.plot(x, f(x, A=1, B=1, C=1))
    
    def update(A = 1, B = 0, C = 0):
        line.set_ydata(f(x,A,B,C))
        fig.canvas.draw_idle()
    
    interact(update, A = (-4,4,0.1), B = (-4,4,0.1), C = (-4,4,0.1));
_box15()

In [ ]:

def bend_coord(angle, coord):
    U = coord_funcs.rotation_matrix(angle)
    _xy = np.dot(coord[:,:2], U)
    _coord = coord.copy() # copy?
    _coord[:,:2] = _xy
    return _coord

def find_correct(atoms_b, coord_b, atoms_a, coord_a):

    coord_a = coord_a.copy()
    coord_a -= rmsdlib.get_cm(atoms_a, coord_a)
    
    coord_b = coord_b.copy()
    coord_b -= rmsdlib.get_cm(atoms_b, coord_b)

    inertia_a = rmsdlib.get_inertia_tensor(atoms_a, coord_a)
    eigval_a, eigvec_a = np.linalg.eig(inertia_a)

    inertia_b = rmsdlib.get_inertia_tensor(atoms_b, coord_b)
    eigval_b, eigvec_b = np.linalg.eig(inertia_b)

    print(eigval_b)

    eigvec_a = eigvec_a.T
    eigvec_a = eigvec_a[np.argsort(eigval_a)]
    eigvec_a = eigvec_a.T
    
    eigvec_b = eigvec_b.T
    eigvec_b = eigvec_b[np.argsort(eigval_b)]
    eigvec_b = eigvec_b.T

    coord_a = np.dot(coord_a, eigvec_a)
    
    # eigvec_b = -eigvec_b
    # if np.linalg.det(eigvec_b) > 0.0:
    #        eigvec_b = -eigvec_b
    
    print(f"Det {np.linalg.det(eigvec_b):.2f}")

    min_rmsd = np.inf
    min_coord = coord_b
    min_review = np.arange(len(atoms_a))

    for mirror in rmsdlib.AXIS_REFLECTIONS:

        vec = np.array(eigvec_b, copy=True)
        vec = vec * mirror.T

        tmp_coord = np.array(coord_b, copy=True)
        tmp_coord = np.dot(tmp_coord, vec)
        tmp_coord -= rmsdlib.get_cm(atoms_b, tmp_coord)

        _review = rmsdlib.reorder_hungarian(atoms_a, atoms_b, coord_a, tmp_coord)

        test_rmsd = rmsdlib.rmsd(tmp_coord[_review], coord_a)
        print(mirror, f"{test_rmsd:.2f}", _review)

        if test_rmsd < min_rmsd:
            min_rmsd = test_rmsd
            min_review = _review
            min_coord = tmp_coord[_review]

    #coord = np.dot(coord, eigvec_)

    return min_coord


def scan_angles():

    _atoms_a = np.array(atoms_a, copy=True)
    _atoms_b = np.array(atoms_a, copy=True)

    _coord_a = np.array(coord_a, copy=True)
    _coord_b = np.array(coord_a, copy=True)
    
    _coord_a -= rmsdlib.get_cm(_atoms_a, _coord_a)
    _coord_b -= rmsdlib.get_cm(_atoms_b, _coord_b)

    inertia_a = rmsdlib.get_inertia_tensor(_atoms_a, _coord_a)
    eigval_a, eigvec_a = np.linalg.eig(inertia_a)

    inertia_b = rmsdlib.get_inertia_tensor(_atoms_b, _coord_b)
    eigval_b, eigvec_b = np.linalg.eig(inertia_b)

    eigvec_a = eigvec_a.T
    eigvec_a = eigvec_a[np.argsort(eigval_a)]
    eigvec_a = eigvec_a.T
    
    eigvec_b = eigvec_b.T
    eigvec_b = eigvec_b[np.argsort(eigval_b)]
    eigvec_b = eigvec_b.T

    _coord_a = np.dot(_coord_a, eigvec_a)
    _coord_b = np.dot(_coord_b, eigvec_b)


    n_rows = 3
    size = 5

    fig = plt.figure(figsize=(size* n_rows, size))
    
    def update(angle):
        coord_ = bend_coord(angle, _coord_b)
        coord_ -= rmsdlib.get_cm(_atoms_b, coord_)

        print("start rmsd", rmsdlib.rmsd(_coord_a, coord_))

        coord_c = find_correct(_atoms_b, coord_, _atoms_a, coord_a)
        
        ax1 = fig.add_subplot(1, 3, 1)
        ax2 = fig.add_subplot(1, 3, 2, sharex=ax1, sharey=ax1)
        ax3 = fig.add_subplot(1, 3, 3, sharex=ax1, sharey=ax1)
        
        ax1.set_xlim(-fig_width, fig_width)
        ax1.set_ylim(-fig_width, fig_width)
        
        plot_eig(ax1, atoms_a, _coord_a)
        plot_eig(ax2, atoms_a, coord_)
        plot_eig(ax3, atoms_a, coord_c)
        
        fig.canvas.draw_idle()
    
    interact(update,angle=(-180, 180, 5));

scan_angles()

In [ ]:
# Plot rotation
fig, ax = plot_funcs.get_plot()

centroid = np.asarray([0, 0])
radius = 1.0

p1 = np.array(centroid)
p1[0] += radius
p1[1] += radius

degrees = 80
step_size = 5
n_steps = int(degrees / step_size)

p2 = p1
for d in range(n_steps):
    U = coord_funcs.rotation_matrix(step_size)
    p3 = np.dot(p2, U)
    plot_funcs.do_arrow(ax, p2, p3, rad=.01)
    p2 = p3

plot_funcs.do_dot(ax, *centroid)
plot_funcs.do_dot(ax, *p1)
plot_funcs.do_dot(ax, *p2)

ax.set_xlim(-1.5, 1.5)
ax.set_ylim(-1.5, 1.5)

pass

In [ ]:
def box91():


    coord = [
        [0,0,0],
        [1,1,0],
        [2,0,0],
    ]
    coord = np.array(coord)
    
    fig, ax = plot_funcs.get_plot()

    fix_width = 2.0
    ax.set_xlim(-fig_width, fig_width)
    ax.set_ylim(-fig_width, fig_width)


    plot_funcs.plot_molecule(ax, ["C", "C", "C"], coord)


box91()

In [ ]:
def box92():


    coord = [
        [0,0,0],
        [1,1,0],
        [2,0,0],
    ]
    coord = np.array(coord)
    atoms = [6,6,6]

    # qml vectors
    parameters = {
        "elements": np.unique(atoms),
        "pad": 3,
        "rcut": 20,
        "acut": 20,
    }

    p_vecs = generate_fchl19(atoms, coord, **parameters)

    # Find non-zero
    s = np.sum(p_vecs, axis=0)
    print(s.shape)
    print(s)

    non_zero_indicies, = np.where(s > 10**-1)

    print(non_zero_indicies)

    p_vecs = p_vecs[:,non_zero_indicies]
    
    

    
    print(p_vecs)

    # Make a KDE?

    
    
    fig, ax = plot_funcs.get_plot()

    ins1 = ax.inset_axes([0.2,0.7,0.4,0.2])
    ins1.imshow(np.expand_dims(p_vecs[0], axis=1), cmap="gray")
    ins1.xaxis.set_ticks([])
    ins1.yaxis.set_ticks([])

    ins2 = ax.inset_axes([0.5,0.7,0.4,0.2])
    ins2.imshow(np.expand_dims(p_vecs[1], axis=1), cmap="gray")
    ins2.xaxis.set_ticks([])
    ins2.yaxis.set_ticks([])

    ins3 = ax.inset_axes([0.7,0.7,0.2,0.2],)
    ins3.imshow(np.expand_dims(p_vecs[2], axis=1), cmap="gray")
    #ins3.axis('off')
    ins3.xaxis.set_ticks([])
    ins3.yaxis.set_ticks([])

    ins3.margins(0.5)
    ins3.patch.set_edgecolor('black')  
    ins3.patch.set_linewidth(1)  

    ax.plot(p_vecs[0])
    ax.plot(p_vecs[1])
    ax.plot(p_vecs[2])
    
    fix_width = 2.0
    
    #ax.set_xlim(-fig_width, fig_width)
    #ax.set_ylim(-fig_width, fig_width)

    #plot_funcs.plot_molecule(ax, ["C", "C", "C"], coord)


box92()